In [1]:
import pyspark
from delta import configure_spark_with_delta_pip
from deltalake import DeltaTable
import pandas as pd
import numpy as np

In [2]:


#create spark context
builder = pyspark.sql.SparkSession.builder.appName("mlops") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
# read data
df_test = spark.read.csv("/Users/alexander/mlops/dataset/test_FD001.txt", sep=" ")
df_train = spark.read.csv("/Users/alexander/mlops/dataset/train_FD001.txt", sep=" ")
df_rul = spark.read.csv("/Users/alexander/mlops/dataset/RUL_FD001.txt", sep=" ",)


# save data in delta lake bronze stage
df_test.write.format("delta").save("delta-lake/bronze/test_data")
df_train.write.format("delta").save("delta-lake/bronze/train_data")
df_rul.write.format("delta").save("delta-lake/bronze/rul_data")





22/01/30 17:34:43 WARN Utils: Your hostname, MacBook-Air-von-Alexander.local resolves to a loopback address: 127.0.0.1; using 192.168.0.9 instead (on interface en0)
22/01/30 17:34:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/alexander/miniforge3/envs/spark/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alexander/.ivy2/cache
The jars for the packages stored in: /Users/alexander/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7944e5d6-9c09-446a-862d-50984786675f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 287ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------


In [3]:
# define column names
cols=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]

In [4]:
# read from deltalake
dt_test = DeltaTable("delta-lake/bronze/test_data")
dt_train = DeltaTable("delta-lake/bronze/train_data")
dt_rul =  DeltaTable("delta-lake/bronze/rul_data")

# convert to dataframe
test = dt_test.to_pandas()
train = dt_train.to_pandas()
test_results = dt_rul.to_pandas()

# set column names
test.columns = cols
train.columns = cols

# set datatypes
test = test.apply(pd.to_numeric)
train = train.apply(pd.to_numeric)



Generate remaining cycle

In [5]:
cols=["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]

read in data

In [6]:
#train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
#test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
#test_results=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)

generate columns for RUL, cycle, rul failed, remaining cycle

In [7]:
test_results = test_results.apply(pd.to_numeric)
test_results.columns=["rul", "null"]

test_results['id']=test_results.index+1
test_results.drop(["null"],axis=1,inplace=True)
rul = pd.DataFrame(test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
test_results['rul_failed']=test_results['rul']+rul['max']
test_results.drop(["rul"],axis=1,inplace=True)
test=test.merge(test_results,on=['id'],how='left')
test["remaining_cycle"]=test["rul_failed"]-test["cycle"]



drop null values and rul failed for test set

In [8]:
df_train=train.drop(["sensor22","sensor23"],axis=1)
df_test=test.drop(["sensor22","sensor23"],axis=1)
df_test.drop(["rul_failed"],axis=1,inplace=True)
df_train['remaining_cycle'] = df_train.groupby(['id'])['cycle'].transform(max)-df_train['cycle']

display columns

In [9]:
train.head()


,id,cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,sensor22,sensor23
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


generate label

In [10]:
# set CYCLES variable to predict if the engine will fail within the next n cycles
CYCLES = 20 # predict if the engine will fail in the next 20 cycles
df_train['label'] = df_train['remaining_cycle'].apply(lambda x: 1 if x <= CYCLES else 0)
df_test['label'] = df_test['remaining_cycle'].apply(lambda x: 1 if x <= CYCLES else 0)

In [11]:
df_train["sensor17"] = pd.to_numeric(df_train["sensor17"], downcast="float")
df_test["sensor17"] = pd.to_numeric(df_test["sensor17"], downcast="float")
df_train["sensor18"] = pd.to_numeric(df_train["sensor18"], downcast="float")
df_test["sensor18"] = pd.to_numeric(df_test["sensor18"], downcast="float")
df_train["sensor19"] = pd.to_numeric(df_train["sensor19"], downcast="float")
df_test["sensor19"] = pd.to_numeric(df_test["sensor19"], downcast="float")


In [12]:
df_train.dtypes

id                   int64
cycle                int64
op1                float64
op2                float64
op3                float64
sensor1            float64
sensor2            float64
sensor3            float64
sensor4            float64
sensor5            float64
sensor6            float64
sensor7            float64
sensor8            float64
sensor9            float64
sensor10           float64
sensor11           float64
sensor12           float64
sensor13           float64
sensor14           float64
sensor15           float64
sensor16           float64
sensor17           float32
sensor18           float32
sensor19           float32
sensor20           float64
sensor21           float64
remaining_cycle      int64
label                int64
dtype: object

save the file

In [13]:
#parquet
#df_train.to_parquet('engine-train.parquet')
#df_test.to_parquet('engine-test.parquet')

#csv
#df_train.to_csv('engine-train.csv')
#df_test.to_csv('engine-test.csv')

In [16]:
# convert pandas dataframes to spark dataframes
df_test_silver = spark.createDataFrame(df_test)
df_train_silver = spark.createDataFrame(df_train)


In [17]:
# save prepared data in delta lake silver stage
df_test_silver.write.format("delta").save("delta-lake/silver/test_data")
df_train_silver.write.format("delta").save("delta-lake/silver/train_data")


22/01/30 18:13:32 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/01/30 18:13:32 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/01/30 18:13:33 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/01/30 18:13:37 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/01/30 18:13:37 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/01/30 18:13:37 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
